<a href="https://colab.research.google.com/github/Tlen33/Genetic-algorithms.-Password-guessing/blob/main/%D0%93%D0%B5%D0%BD%D0%B5%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%9F%D0%BE%D0%B4%D0%B1%D0%BE%D1%80_%D0%BF%D0%B0%D1%80%D0%BE%D0%BB%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Основные этапы работы алгоритма

1.   обработка первой популяции
2.   оценка популяции
3.   получение элитарных ботов
4.   получение лучших ботов
5.   получение лучших ботов и добавление к ним элитных, для сохранения лучших результатов
6.   список ботов для следующего отбора







In [ ]:
# Импорт библиотек

import string
import random
import pandas

In [ ]:
# Основные параметры

# Стандартная фраза
password = "The Quick Brown Fox Jumps Over Lazy Dog" 

# Количество ботов начальной популяции (это будут боты со случайными генами)
number_of_bots = 100 

# доля сохраненных элитных ботов (они нужны, чтобы сохранить лучший результат)
elite_bots_to_keep = .01 

# доля лучших ботов (они нужны, чтобы улучшать результат)
best_bots_to_keep = .2 

# минимальное кол-во создаваемых ботов в новой популяции
quantity_of_bots_in_created_generations = 1000 

# коэффициент мутации - вероятность гена мутировать (получить случайное значение отличное от родителей)
mutation_rate = .8 

In [ ]:
# Основные функции

def get_random_letter():
  """
  Создание случайной буквы
  """
  return random.choice(string.ascii_letters + ' ') # возвращаем одну случайную букву

In [ ]:
def create_bots(number_of_bot_to_create):
  """
  Создание популяции ботов
  """
  # создание пустого списка ботов для формирование случайной фразы (каждая буква - это аналог гена)
  bots = [] 
  
  # создаем цикл для наполнения ВСЕХ ботов буквами (генами)
  for _ in range(number_of_bot_to_create):

      # обнуляем бота в начале каждого цикла
      bot = "" 

      # создаем цикл для наполнения ОДНОГО бота буквами (генами)
      for _ in range(len(password)):

          # добавляем одну букву (ген) на один цикл
          bot += get_random_letter() 

      # добавляем созданного бота в список ботов
      bots.append(bot) 

  # возвращаем список ботов
  return bots 

In [ ]:
def estimate_bot(bot: str):
  """
  Оценка 1 бота. 
  Аналог лосс функции в нейронной сети.
  """
  error = len(password) # первоначальное значение ошибки для каждого бота, она равна кол-ву букв, которые не совпали. 
                        # Худший вариант - не совпали все, тогда ошибка равна длине фразы. 
                        # Лучший вариант - совпали все, тогда, алгоритм заканчивает работу.

  # в цикле проходим по каждой букве (гену), сравниваем с каждой буквой в реальном пароле. При каждом совпадении вычитаем единицу из ошибки
  for i in range(len(password)):
      if bot[i] == password[i]: # проверяем совпадают ли буквы
          error -= 1 # вычитаем единицу. если совпадают
  return bot, error

In [ ]:
def estimate_generation(generation):
  """
  Оценка популяции вцелом
  """
  bot_dict = {} # создаем пустой словарь, в который будем собирать бота и его качество/ценность, расчитанныую с помощью лосс-функции (estimate_bot(bot))
  # Проходим по каждому боту в популяции, вычисляя ошибку.
  for created_bot in generation:
      bot, error = estimate_bot(bot=created_bot) # вычислем ошибку каждого бота в цикле (каждый цикл новый бот)
      # дописываем словарь соответствущими расчитаными ошибками и самим ботом
      bot_dict.update({ 
          bot: [error],
      })
  
  bot_df = pandas.DataFrame(bot_dict).T # меняем строки и столбцы местами. Такова оссобенность формирования датафреймов в пандасе.
  bot_df.columns = ["error"] # переименовываем колонку с ошибками (это необязательно, но удобнее в случае дальнейшей работы)
  sorted_df = bot_df.sort_values(by=["error"], ascending=True) # сортируем датафрейм по ошибке, чтобы сверху оказались лучшие боты.
  return sorted_df # возвращаем отсортированный список

In [ ]:
def keep_elite_bots(df):
  """
  Сохранение лучших ботов за все популяции. Необхдоимо для сохранения качества популяции, нужны в том случае,
  если все созданные боты оказались хуже своих родителей, тогда лучшие боты сохранять качество популяции.
  """
  return df.iloc[:int(number_of_bots * elite_bots_to_keep), :].index.to_list()

In [ ]:
def keep_best_bots(df: pandas.DataFrame):
  """
  Получение лучших ботов из текущей популяции. Аналог естественного отбора. Выживают те, кто ближе всего к оптимальному результату.
  """
  return df.iloc[:int(number_of_bots * best_bots_to_keep), :].index.to_list()

In [ ]:
def divide_bots(survived_bots):
  """
  Разделение ботов на родителя 1 и родителя 2
  """
  parent_2 = [] # создаем список, в котором будем хранить одного из родителей.
  # Необходимо разделить на 2 равные части, так как родителей двое. Далее мы проверим четность, если на 2 делится - все хорошо, если не делится. то удаляем последний элемент
  if len(survived_bots) % 2 != 0:
      survived_bots.remove(survived_bots[-1]) # удаление последнего элемента, если значение четное
  parent_1 = random.sample(survived_bots, int(len(survived_bots) / 2)) # создаем список, в который будем сохранять половину ботов (это будет "родитель 1").
  # проходим циклом по всем ботам, проверяем являются ли они "родителем 1", если нет, то добавляем их в список "родитель 2"
  for bot in survived_bots:
      if bot not in parent_1: # проверяем являются ли они "родителем 1"
          parent_2.append(bot) # добавляем их в список "родитель 2"
  return parent_1, parent_2 # возвращаем оба списка (список "родитель 1" и "родитель 2")

In [ ]:
def create_new_generation(survived_bots):
  """
  Создание новой популяции. Бот состоит из 32 генов, мы случайно выбираем половину генов одного родителя, половину второго и создаем потомка.
  Так повторяем многократно. Таким образом мы создаем немного другую популяцию с одной стороны, но измененную с другой. Следующим шагом мы оцениваем популяцию,
  оставляем лучших, из опять объединяем между собой. Это неочевидно, но в какой-то момент на определенном месте может оказаться неверный ген, но он может оказаться
  у всех представителей популяции. Например, на первом месте вместо "T" будет "d", тогда у какого бы из родителей мы бы не взяли ген, он останется неверным,
  чтобы исправить ситуацию введен механизм мутации, он случайно выбирает букву вне зависимости от генов родителей.
  """
  parent_list_1, parent_list_2 = divide_bots(survived_bots=survived_bots) # разделение популяции на два списка "родитель 1" и "родитель 2"
  gene_index = random.sample(range(32), 16) # случайно выбираем индексы, которые потомок получит от родителя 1, остальные он получит от родителя 2
  child_list = [] # создаем пустой список, в который будем добавлять потомков
  # проверяем не превысило ли количество потомков некоторую величину. В нашем случае 1000, но значение может быть любое
  while len(child_list) < quantity_of_bots_in_created_generations: 
      # создаем цикл, в котором проходим по спискам родителей. Получаем индекс родителя, далее по этому индексу получаем конкретного родителя.
      for parent_index in range(len(parent_list_1)):
          child = "" # обнуляем гены потомка
          # создаем цикл, в которым проходим по каждой букве (гену) каждого из родителя, если этот индекс есть в списке тех, которые потомок должен получить
          # от родителя 1, то потомок получает букву (ген) от родителя 1, если нет - то наоборот.
          for i in range(len(password)):
              if i in gene_index: # проверяем находится ли индекс в списке генов, которые должны быть получено от родителя 1
                  # Проверяем мутировал ли ген
                  if random.uniform(0, 1) < .8:
                      child += parent_list_1[parent_index][i] # если не мутировал, но букву(ген) потомок получает отродителя 1
                  else:
                      child += get_random_letter() # если мутировал, то получает случайную букву
              else:
                  # Проверяем мутировал ли ген
                  if random.uniform(0, 1) < .8:
                      child += parent_list_2[parent_index][i] # если не мутировал, но букву(ген) потомок получает отродителя 1
                  else:
                      child += get_random_letter() # если мутировал, то получает случайную букву
          child_list.append(child) # добавление потомка в список потомков
  return child_list # возвращение списка потомков

In [ ]:
def print_current_results(sorted_df, to_print):
  """
  Выведение текущих результатов на экран
  """
  # Проверяем нужно ли выводить текущие результаты на экран
  if to_print: 
    # если нужно выводить
    error = sorted_df.iloc[0, :].values[0] # получаем значение ошибки
    text = sorted_df.iloc[:, :].index.to_list()[0] # получаем текущий текст
    print(error, text) # выводим на печать ошибку и текст
  else:
    # усли не нужно выводить
    pass # ничего не делаем

In [ ]:
def print_final_result(sorted_df):
  """
  Выведение на печать финального результата
  """
  text = sorted_df.iloc[:, :].index.to_list()[0] # получаем текущий текст
  print(f"\n\nDone: correct_text is {text}") # выведение его на экран

## Запуск генетического алгоритма

In [ ]:
# Генетический алгоритм

# Создание первой рандомной популяции
first_generation = create_bots(number_of_bot_to_create=number_of_bots)

# оценка популяции
df = estimate_generation(generation=first_generation)

# получение элитарных ботов
elite = keep_elite_bots(df=df)

# получение лучших ботов
best_bots = keep_best_bots(df=df)

# получение лучших ботов и добавление к ним элитных, для сохранения лучших результатов
survived_bots = sorted(list(set(elite + best_bots)))

# список ботов для следующего отбора
child_list = create_new_generation(survived_bots=survived_bots)

In [ ]:
# Основная логика генетического алгоритма
# инициализация начального значения ошибки, которое равно кол-ву букв
error = len(password)
epoch = 0

while epoch < 1000:
  # оценка текущей популяции
  sorted_df = estimate_generation(generation=child_list)
  # выведение на экран текущих результатов, если нужно
  print_current_results(sorted_df=sorted_df, to_print=True)
  # сохранение элитных ботов для сохранения лучших результатов
  elite = keep_elite_bots(df=sorted_df)
  # получение лучших ботов из текущей популяции
  best_bots = keep_best_bots(df=sorted_df)
  # получение выживших ботов
  survived_bots = sorted(list(set(best_bots)))
  # создание новой популяции (текущая + лучшая из предыдущих поколений)
  child_list = create_new_generation(survived_bots=survived_bots) + elite
  # расчет ошибки
  error = sorted_df.iloc[0, :].values[0]

# Вывод на печать распознанного текста
print_final_result(sorted_df=sorted_df)

Выходные данные были обрезаны до нескольких последних строк (5000).
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Quick Brown zox Jumss Over LazU Dog
3 The Qu